[View in Colaboratory](https://colab.research.google.com/github/hschoi1/rich_latent/blob/master/Anomaly_Detection.ipynb)

In [0]:
# Dependency imports
from absl import flags
import numpy as np
from six.moves import urllib
import tensorflow as tf
import functools
import itertools
import os
import tensorflow_probability as tfp
from sklearn.metrics import roc_auc_score, average_precision_score
tfd = tfp.distributions
tfb = tfp.bijectors
import matplotlib.pyplot as plt
import pdb
import pandas as pd

In [0]:
# Flags
# IMAGE_SHAPE = (28, 28, 1)
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:    
       FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags.DEFINE_float(
    "learning_rate", default=0.001, help="Initial learning rate.")
flags.DEFINE_integer(
    "max_steps", default=2001, help="Number of training steps to run.")
flags.DEFINE_integer(
    "latent_size",
    default=16,
    help="Number of dimensions in the latent code (z).")
flags.DEFINE_integer("base_depth", default=32, help="Base depth for layers.")
flags.DEFINE_string(
    "activation",
    default="leaky_relu",
    help="Activation function for all hidden layers.")
flags.DEFINE_integer(
    "batch_size",
    default=32,
    help="Batch size.")
flags.DEFINE_integer(
    "n_samples", default=16, help="Number of samples to use in encoding.")
flags.DEFINE_bool(
    "use_NF",
    default = True,
    help = "If False, normalizing flows are not applied")
flags.DEFINE_integer(
    "mixture_components",
    default=100,
    help="Number of mixture components to use in the prior. Each component is "
         "a diagonal normal distribution. The parameters of the components are "
         "intialized randomly, and then learned along with the rest of the "
         "parameters. If `analytic_kl` is True, `mixture_components` must be "
         "set to `1`.")
flags.DEFINE_integer("n_flows", default=6, help="Number of Normalizing Flows")
flags.DEFINE_float("elbo_threshold", default=5.0, help="anomaly threshold for whole elbo")
flags.DEFINE_bool(
    "analytic_kl",
    default=False,
    help="Whether or not to use the analytic version of the KL. When set to "
         "False the E_{Z~q(Z|X)}[log p(Z)p(X|Z) - log q(Z|X)] form of the ELBO "
         "will be used. Otherwise the -KL(q(Z|X) || p(Z)) + "
         "E_{Z~q(Z|X)}[log p(X|Z)] form will be used. If analytic_kl is True, "
         "then you must also specify `mixture_components=1`.")
flags.DEFINE_string(
    "data_dir",
    default=os.path.join(os.getenv("TEST_TMPDIR", "/tmp"), "vae/data"),
    help="Directory where data is stored (if using real data).")
flags.DEFINE_string(
    "model_dir",
    default=os.path.join(os.getenv("TEST_TMPDIR", "/tmp"), "vae/"),
    help="Directory to put the model's fit.")
flags.DEFINE_integer(
    "viz_steps", default=500, help="Frequency at which to save visualizations.")


flags.DEFINE_bool(
    "delete_existing",
    default=False,
    help="If true, deletes existing `model_dir` directory.")


FLAGS = flags.FLAGS


In [0]:

ROOT_PATH = "http://www.cs.toronto.edu/~larocheh/public/datasets/binarized_mnist/"
FILE_TEMPLATE = "binarized_mnist_{split}.amat"

def download(directory, filename):
  """Download a file."""
  filepath = os.path.join(directory, filename)
  if tf.gfile.Exists(filepath):
    return filepath
  if not tf.gfile.Exists(directory):
    tf.gfile.MakeDirs(directory)
  url = os.path.join(ROOT_PATH, filename)
  print("Downloading %s to %s" % (url, filepath))
  urllib.request.urlretrieve(url, filepath)
  return filepath


def static_mnist_dataset(directory, split_name):
  """Return binary static MNIST tf.data.Dataset."""
  amat_file = download(directory, FILE_TEMPLATE.format(split=split_name))
  dataset = tf.data.TextLineDataset(amat_file)
  str_to_arr = lambda string: np.array([char == "1" for char in string.split()])

  def _parser(s):
    booltensor = tf.py_func(str_to_arr, [s], tf.bool)
    reshaped = tf.reshape(booltensor, [28, 28, 1])
    return tf.to_float(reshaped), tf.constant(0, tf.int32)

  return dataset.map(_parser)

def build_fake_input_fns(batch_size, eval_repeat=1, image_shape=(28, 28, 1)):
  """Build fake MNIST-style data for unit testing."""
  dataset = tf.data.Dataset.from_tensor_slices(
      np.random.rand(batch_size, *image_shape).astype("float32")).map(
          lambda row: (row, 0)).batch(batch_size)

  train_input_fn = lambda: dataset.repeat().make_one_shot_iterator().get_next()
  eval_input_fn = lambda: dataset.repeat(eval_repeat).make_one_shot_iterator().get_next()
  return train_input_fn, eval_input_fn


def build_mnist_input_fns(data_dir, batch_size):
  """Build an Iterator switching between train and heldout data."""

  # Build an iterator over training batches.
  training_dataset = static_mnist_dataset(data_dir, "train")
  training_dataset = training_dataset.shuffle(50000).repeat().batch(batch_size)
  train_input_fn = lambda: training_dataset.make_one_shot_iterator().get_next()

  # Build an iterator over the heldout set.
  eval_dataset = static_mnist_dataset(data_dir, "valid")
  eval_dataset = eval_dataset.batch(batch_size)
  eval_input_fn = lambda: eval_dataset.make_one_shot_iterator().get_next()

  return train_input_fn, eval_input_fn

def build_credit_dataset(batch_size):
    df = pd.read_csv('drive/Colab Notebooks/creditcard.csv.zip',header=0, sep=',', quotechar='"')
    normal = df[df.Class == 0]
    anormal = df[df.Class == 1]  


    X_index = df.columns[1:-2]
    normal_features = normal[X_index]
    normal_time_amount = normal[['Time','Amount']]
    normal_time_amount = (normal_time_amount- normal_time_amount.mean())/normal_time_amount.std()
    normal_X = pd.concat([normal_features, normal_time_amount], axis=1)
    normal_X = normal_X.as_matrix()
    normal_X = normal_X.astype(np.float32)

    # split into train/test splits
    #choices = np.random.choice(normal_X.shape[0],1000)
    n_anormal = anormal.shape[0]  # 492
    np.random.shuffle(normal_X)
    test_normal_X = normal_X[:n_anormal]
    #test_normal_X = normal_X[choices]
    train_normal_X = normal_X[n_anormal:]
    
    anormal_features = anormal[X_index]
    anormal_time_amount = anormal[['Time','Amount']]
    anormal_time_amount = (anormal_time_amount- anormal_time_amount.mean())/anormal_time_amount.std()
    anormal_X = pd.concat([anormal_features, anormal_time_amount], axis=1)
    anormal_X = anormal_X.as_matrix()
    anormal_X = anormal_X.astype(np.float32)
    
    dataset = tf.data.Dataset.from_tensor_slices((train_normal_X, np.zeros(train_normal_X.shape[0])))
    dataset = dataset.shuffle(buffer_size=256)
    dataset = dataset.repeat()      
    dataset = dataset.batch(batch_size)   
    
    
    eval_labels = np.concatenate([np.zeros(n_anormal), np.ones(n_anormal)])
    eval_dataset=tf.data.Dataset.from_tensor_slices((np.concatenate([test_normal_X, anormal_X], axis=0), eval_labels))
    #eval_labels = np.zeros(test_normal_X.shape[0])
    #eval_dataset = tf.data.Dataset.from_tensor_slices((test_normal_X, eval_labels))
    
    
    eval_dataset = eval_dataset.batch(batch_size)

    train_input_fn = lambda: dataset.repeat().make_one_shot_iterator().get_next()
    eval_input_fn = lambda: eval_dataset.make_one_shot_iterator().get_next()
    return train_input_fn, eval_input_fn  
  

def build_keras_dataset(keras_dataset, batch_size, expand_last_dim=False):
  (x_train, y_train), (x_test, y_test) = keras_dataset.load_data()
  
  if expand_last_dim:
    x_train = np.expand_dims(x_train, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)
  
  x_train = x_train.astype(np.float32)/255.
  x_test = x_test.astype(np.float32)/255.
  train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
  eval_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
  
  train_input_fn = lambda: train_dataset.repeat().make_one_shot_iterator().get_next()
  eval_input_fn = lambda: eval_dataset.make_one_shot_iterator().get_next()
  return train_input_fn, eval_input_fn

def get_dataset(dataset_name, batch_size):
  # unified dataset loading fn
  if dataset_name == 'mnist':
    return build_mnist_input_fns(FLAGS.data_dir, batch_size)
  elif dataset_name == 'fashion_mnist':
    return build_keras_dataset(tf.keras.datasets.fashion_mnist, batch_size)
  elif dataset_name == 'noise':
    return build_fake_input_fns(batch_size, eval_repeat=10)
  elif dataset_name == 'cifar10':
    return build_keras_dataset(tf.keras.datasets.cifar10, batch_size)
  elif dataset_name =='credit_card':
    return build_credit_dataset(batch_size)
    

In [0]:


  
def init_once(x, name):
  return tf.get_variable(name, initializer=x, trainable=False)

def make_arflow(z_dist, latent_size, n_flows, hidden_size=(512, 512), invert=False):
    chain = list(itertools.chain.from_iterable([
        tfb.MaskedAutoregressiveFlow(
            shift_and_log_scale_fn=tfb.masked_autoregressive_default_template(
                hidden_size)),
        tfb.Permute(init_once(np.random.permutation(latent_size), 'permute_%d' % i)),
    ] for i in range(n_flows)))
    return tfd.TransformedDistribution(distribution=z_dist, bijector=tfb.Chain(chain[:-1]))

def make_NF_prior(latent_size, n_flows):
    return make_arflow(tfd.MultivariateNormalDiag(
        loc=tf.zeros([latent_size], dtype=tf.float32),
        scale_diag = tf.ones([latent_size], dtype=tf.float32)),
                       latent_size, n_flows)


  
  

def make_mixture_prior(latent_size, mixture_components):
  """Create the mixture of Gaussians prior distribution. Prior is learned.
  Args:
    latent_size: The dimensionality of the latent representation.
    mixture_components: Number of elements of the mixture.
  Returns:
    random_prior: A `tf.distributions.Distribution` instance
      representing the distribution over encodings in the absence of any
      evidence.
  """
  if mixture_components == 1:
    # See the module docstring for why we don't learn the parameters here.
    return tfd.MultivariateNormalDiag(
        loc=tf.zeros([latent_size]),
        scale_identity_multiplier=1.0)
  else:
    loc = tf.get_variable(name="loc", shape=[mixture_components, latent_size])
    raw_scale_diag = tf.get_variable(
        name="raw_scale_diag", shape=[mixture_components, latent_size])
    mixture_logits = tf.get_variable(
        name="mixture_logits", shape=[mixture_components])

    return tfd.MixtureSameFamily(
        components_distribution=tfd.MultivariateNormalDiag(
            loc=loc,
            scale_diag=tf.nn.softplus(raw_scale_diag)),
        mixture_distribution=tfd.Categorical(logits=mixture_logits),
        name="prior")


In [0]:
# Credit Card Model


def _softplus_inverse(x):
  """Helper which computes the function inverse of `tf.nn.softplus`."""
  return tf.log(tf.expm1(x))

def make_encoder(activation, latent_size, base_depth):
  """Create the encoder function.
  Args:
    activation: Activation function to use.
    latent_size: The dimensionality of the encoding.
    base_depth: The lowest depth for a layer.
  Returns:
    encoder: A `callable` mapping a `Tensor` of images to a
      `tf.distributions.Distribution` instance over encodings.
  """

  encoder_net = tf.keras.Sequential([
      tf.keras.layers.Dense(2 * latent_size, activation=None),
  ])

  def encoder(images):
    net = encoder_net(images)
    return tfd.MultivariateNormalDiag(
        loc=net[..., :latent_size],
        scale_diag=tf.nn.softplus(net[..., latent_size:] +
                                  _softplus_inverse(1.0)),
        name="code")

  return encoder


def make_decoder(activation, latent_size, output_shape, base_depth):
  """Create the decoder function.
  Args:
    activation: Activation function to use.
    latent_size: Dimensionality of the encoding.
    output_shape: The output image shape.
    base_depth: Smallest depth for a layer.
  Returns:
    decoder: A `callable` mapping a `Tensor` of encodings to a
      `tf.distributions.Distribution` instance over images.
  """

  decoder_net = tf.keras.Sequential([
      tf.keras.layers.Dense(30),
  ])

  def decoder(codes):
    #pdb.set_trace()
    original_shape = tf.shape(codes)
    # Collapse the sample and batch dimension and convert to rank-4 tensor for
    # use with a convolutional decoder network.
    codes = tf.reshape(codes, (-1, 1, 1, latent_size))
    logits = decoder_net(codes)
    logits = tf.reshape(
        logits, shape=tf.concat([original_shape[:-1], output_shape], axis=0))
    return tfd.Independent(
        tfd.Bernoulli(logits=logits),
        reinterpreted_batch_ndims=len(output_shape),
        name="image")

  return decoder



def anomaly_model_fn(features, labels, mode, params, config):
  """Build the model function for use in an estimator.
  Arguments:
    features: The input features for the estimator.
    labels: The labels, unused here.
    mode: Signifies whether it is train or test or predict.
    params: Some hyperparameters as a dictionary.
    config: The RunConfig, unused here.
  Returns:
    EstimatorSpec: A tf.estimator.EstimatorSpec instance.
  """
  #del labels, config
  predictions = {}

  if params["analytic_kl"] and params["latent_size"] != 1:
    raise NotImplementedError(
        "Using `analytic_kl` is only supported when `mixture_components = 1` "
        "since there's no closed form otherwise.")
    
  encoder = make_encoder(params["activation"],
                         params["latent_size"],
                         params["base_depth"])
  
  image_shape = features.get_shape().as_list()[1:]
  decoder = make_decoder(params["activation"],
                         params["latent_size"],
                         image_shape,
                         params["base_depth"])
  if params['use_NF']:
    latent_prior = make_NF_prior(params["latent_size"],params["n_flows"])
  else:
    latent_prior = make_mixture_prior(params["latent_size"],
                                      params["mixture_components"])

  #pdb.set_trace()
  
  approx_posterior = encoder(features)
  approx_posterior_sample = approx_posterior.sample(params["n_samples"])
  decoder_likelihood = decoder(approx_posterior_sample)


  # `distortion` is just the negative log likelihood.
  distortion = -decoder_likelihood.log_prob(features)
  avg_distortion = tf.reduce_mean(distortion)
  tf.summary.scalar("distortion", avg_distortion)
  
  
  if params["analytic_kl"]:
    raise ValueError('Not Completely Implemented!')
    rate = tfd.kl_divergence(approx_posterior, latent_prior)
  else:
    rate = (approx_posterior.log_prob(approx_posterior_sample)
            - latent_prior.log_prob(approx_posterior_sample))
  avg_rate = tf.reduce_mean(rate)
  tf.summary.scalar("rate", avg_rate)

  elbo_local = -(rate + distortion)

  elbo = tf.reduce_mean(elbo_local)
  loss = -elbo
  tf.summary.scalar("elbo", elbo)
  
  # negative log-likelihood of encoded inputs under likelihood model p(x|z)
  # lower is better
  predictions['distortion'] = distortion 
  predictions['rate'] = rate
  predictions['elbo'] = elbo_local

  importance_weighted_elbo = tf.reduce_mean(
      tf.reduce_logsumexp(elbo_local, axis=0) -
      tf.log(tf.to_float(params["n_samples"])))
  tf.summary.scalar("elbo/importance_weighted", importance_weighted_elbo)


  # Perform variational inference by minimizing the -ELBO.
  global_step = tf.train.get_or_create_global_step()
  learning_rate = tf.train.cosine_decay(0.0001, global_step,
                                        params["max_steps"])
  tf.summary.scalar("learning_rate", learning_rate)
  optimizer = tf.train.AdamOptimizer(learning_rate)
  train_op = optimizer.minimize(loss, global_step=global_step)

  # estimator predictions for inference + visualization
  predictions['approx_posterior_mean'] = approx_posterior.mean()
  predictions['approx_posterior_stddev'] = approx_posterior.scale.diag
  
  # adversarial perturbation
  grad, = tf.gradients(loss, features)
  adversarial_example = features - .1 * tf.sign(grad) # optimize the gibberish to minimize loss.
  predictions['adversarial_example'] = adversarial_example
  #pdb.set_trace()
  elbo_local.set_shape((FLAGS.n_samples, None))
  elbo_local_mean = tf.reduce_mean(elbo_local,axis=0)
  predictions['elbo_local_mean'] = elbo_local_mean
  elbo_local_mean = tf.sigmoid(elbo_local_mean)
  predictions['sigmoid'] = elbo_local_mean
  mask = tf.greater(elbo_local_mean,params['elbo_threshold'])
  predictions['class'] = tf.cast(mask, tf.int32) 
  #elbo_local_mean = tf.clip_by_value(elbo_local_mean, 0.0,1.0)
  #thresholds = [0.0, 0.5, 1.0]
  thresholds = np.arange(0.0,0.2,0.01).tolist()
  
  
  

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(
      mode=mode,
       eval_metric_ops = {
          "elbo": tf.metrics.mean(elbo),
          "elbo/importance_weighted": tf.metrics.mean(importance_weighted_elbo),
          "rate": tf.metrics.mean(avg_rate),
          "distortion": tf.metrics.mean(avg_distortion),
      },
      predictions=predictions
  )
  
  labels = tf.cast(labels, tf.int32)
  
  (metric_tensor1, update_op1) = tf.metrics.true_positives_at_thresholds(labels=labels, predictions=elbo_local_mean,thresholds =thresholds)
  (metric_tensor2, update_op2) = tf.metrics.true_negatives_at_thresholds(labels=labels, predictions=elbo_local_mean,thresholds =thresholds)
  (metric_tensor3, update_op3) = tf.metrics.false_positives_at_thresholds(labels=labels, predictions=elbo_local_mean, thresholds=thresholds)
  (metric_tensor4, update_op4) = tf.metrics.false_negatives_at_thresholds(labels=labels, predictions=elbo_local_mean, thresholds=thresholds)

  summary_hook = tf.train.SummarySaverHook(
    save_steps=100,
    output_dir=FLAGS.model_dir,
    summary_op=tf.summary.merge_all())
  
  return tf.estimator.EstimatorSpec(
      mode=mode,
      loss=loss,
      train_op=train_op,
       eval_metric_ops = {
          "elbo": tf.metrics.mean(elbo),
          "elbo/importance_weighted": tf.metrics.mean(importance_weighted_elbo),
          "rate": tf.metrics.mean(avg_rate),
          "distortion": tf.metrics.mean(avg_distortion),
           "TP": (tf.cast(metric_tensor1, tf.float32), tf.cast(update_op1, tf.float32)),
           "TN": (tf.cast(metric_tensor2, tf.float32), tf.cast(update_op2, tf.float32)),
           "FP": (tf.cast(metric_tensor3, tf.float32), tf.cast(update_op3, tf.float32)),
           "FN": (tf.cast(metric_tensor4, tf.float32), tf.cast(update_op4, tf.float32)),
           "auc": tf.metrics.auc(labels=labels, predictions=elbo_local_mean),
      },
      predictions=predictions, training_hooks=[summary_hook]
  )




In [0]:
#Train Credit Card Model
def statistics(labels, predictions):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for i in range(len(labels)):
        if labels[i] == 1 and predictions[i] == 1:
            TP += 1
        elif labels[i] == 0 and predictions[i] == 1:
            FP += 1
        elif labels[i] == 0 and predictions[i] == 0:
            TN += 1
        elif labels[i] == 1 and predictions[i] == 0:
            FN += 1
    return TP,FP,TN,FN

def print_stats(values, truth, thresholds, name):
  for threshold in thresholds:
    predictions = np.zeros(values.shape[0])
    predictions[np.argwhere(values > threshold)] = 1
    stats = statistics(truth, predictions)
    print(name, str(threshold), ":  TP:",str(stats[0])," FP:",str(stats[1])," TN:",str(stats[2])," FN:",str(stats[3]))
    
  
def main(argv):
  ensemble_elbos = []
  ensemble_posterior_means = []
  ensemble_posterior_vars = []
  M=5 # number of models in ensemble
  for i in range(M):
    params = FLAGS.flag_values_dict()
    params["activation"] = getattr(tf.nn, params["activation"])

    #FLAGS.model_dir = "/usr/local/google/home/ejang/tmp/cifar10/vae%d" % i
    FLAGS.model_dir = "drive/Colab Notebooks/creditcard/vae%d" % i

    if FLAGS.delete_existing and tf.gfile.Exists(FLAGS.model_dir):
      tf.logging.warn("Deleting old log directory at {}".format(FLAGS.model_dir))
      tf.gfile.DeleteRecursively(FLAGS.model_dir)
    tf.gfile.MakeDirs(FLAGS.model_dir)

    train_input_fn, eval_input_fn = get_dataset('credit_card', FLAGS.batch_size)
    
    estimator = tf.estimator.Estimator(
        anomaly_model_fn,
        params=params,
        config=tf.estimator.RunConfig(
            model_dir=FLAGS.model_dir,
            save_checkpoints_steps=FLAGS.viz_steps,
        ),
    )
    
    
    for _ in range(FLAGS.max_steps // FLAGS.viz_steps):    
      estimator.train(train_input_fn, steps=FLAGS.viz_steps)
      eval_results = estimator.evaluate(eval_input_fn)
      print("Evaluation_results:\n\t%s\n" % eval_results)
    
    
    batch_results_ = list(estimator.predict(eval_input_fn,predict_keys=['elbo_local_mean', 'sigmoid', 'approx_posterior_mean', 'approx_posterior_stddev']))
    elbo_local_mean = np.array([b['elbo_local_mean'] for b in batch_results_])
    mean = np.array([b['approx_posterior_mean'] for b in batch_results_])
    stddev = np.array([b['approx_posterior_stddev'] for b in batch_results_])
    #sigmoid = np.array([b['sigmoid'] for b in batch_results_])
    ensemble_elbos.append(elbo_local_mean)
    ensemble_posterior_means.append(mean)
    ensemble_posterior_vars.append(stddev**2)
  
  ensemble_elbos = np.array(ensemble_elbos)
  ensemble_posterior_means = np.array(ensemble_posterior_means)
  ensemble_posterior_vars = np.array(ensemble_posterior_vars)
  
  truth = np.concatenate([np.zeros(492), np.ones(492)])  
  
  
  #single model - last one
  
  
  f, axes = plt.subplots(1, 2, figsize=(5, 5))
  #bins0 = np.linspace(0.0,8000,1000)
  axes[0].hist(elbo_local_mean[:492])
  axes[0].set_xlabel("single elbo from valid data")
  axes[1].hist(elbo_local_mean[492:])
  axes[1].set_xlabel("single elbo from anomlies")
  
  single_elbo = elbo_local_mean
  single_elbo_clipped = np.clip(single_elbo,-150,150)
  max_single_elbo = 300
  single_elbo_clipped = single_elbo_clipped/max_single_elbo + 0.5
  auroc_score = roc_auc_score(y_true=truth, y_score=single_elbo_clipped)  
  ap_score = average_precision_score(y_true=truth, y_score=single_elbo_clipped)
  print("using single elbo,  AUROC: ",str(auroc_score),"  AP: ",str(ap_score))
  
  
  
  
  # ensemble
  #pdb.set_trace()
  ensemble_elbo_mean = np.mean(ensemble_elbos, axis=0)
  ensemble_elbo_mean_clipped = np.clip(ensemble_elbo_mean,-200,200)
  max_ensemble_elbo_mean = 400
  ensemble_elbo_mean_clipped = ensemble_elbo_mean_clipped/max_ensemble_elbo_mean + 0.5
  
  
  ensemble_elbo_var = np.var(ensemble_elbos, axis=0)
  ensemble_elbo_var_clipped = np.clip(ensemble_elbo_var,0,10000)
  max_ensemble_elbo_var = 10000
  ensemble_elbo_var_clipped = ensemble_elbo_var_clipped/max_ensemble_elbo_var
  
  ensemble_posterior_means_mean = np.mean(np.mean(ensemble_posterior_means,axis=2),axis=0)
  ensemble_posterior_means_mean_clipped = np.clip(ensemble_posterior_means_mean, -1.0,1.0)
  max_ensemble_posterior_means_mean = 2.0
  ensemble_posterior_means_mean_clipped = ensemble_posterior_means_mean_clipped/max_ensemble_posterior_means_mean + 0.5
  
  ensemble_posterior_means_var = np.var(np.mean(ensemble_posterior_means,axis=2), axis=0)
  ensemble_posterior_means_var_clipped = np.clip(ensemble_posterior_means_var,0.0,3.0)
  max_ensemble_posterior_means_var = 3.0
  ensemble_posterior_means_var_clipped =  ensemble_posterior_means_var_clipped/ max_ensemble_posterior_means_var
    
  ensemble_posterior_vars_mean =  np.mean(np.mean(ensemble_posterior_vars,axis=2),axis=0)
  ensemble_posterior_vars_mean_clipped = np.clip(ensemble_posterior_vars_mean,0.0,6.0)
  max_ensemble_posterior_vars_mean = 6.0
  ensemble_posterior_vars_mean_clipped = ensemble_posterior_vars_mean_clipped/max_ensemble_posterior_vars_mean
  
  ensemble_posterior_vars_var =  np.var(np.mean(ensemble_posterior_vars,axis=2),axis=0)
  ensemble_posterior_vars_var_clipped = np.clip(ensemble_posterior_vars_var,0,20.0)
  max_ensemble_posterior_vars_var = 20.0
  ensemble_posterior_vars_var_clipped = ensemble_posterior_vars_var_clipped/max_ensemble_posterior_vars_var
  
  
  
  
  auroc_score = roc_auc_score(y_true=truth, y_score=ensemble_elbo_mean_clipped)  
  ap_score = average_precision_score(y_true=truth, y_score=ensemble_elbo_mean_clipped)
  print("using ensemble_elbo_mean,  AUROC: ",str(auroc_score),"  AP: ",str(ap_score))
  
  #ensemble_elbo_var_thresholds = np.arange(0,500,25)
  auroc_score = roc_auc_score(y_true=truth, y_score=ensemble_elbo_var_clipped)  
  ap_score = average_precision_score(y_true=truth, y_score=ensemble_elbo_var_clipped)
  print("using ensemble_elbo_var,  AUROC: ",str(auroc_score),"  AP: ",str(ap_score))
  
  #print_stats(ensemble_elbo_var, truth, ensemble_elbo_var_thresholds, "ensemble elbo var")
  
 
  auroc_score = roc_auc_score(y_true=truth, y_score=ensemble_posterior_means_mean_clipped)  
  ap_score = average_precision_score(y_true=truth, y_score=ensemble_posterior_means_mean_clipped)
  print("using ensemble mean of posterior mean,  AUROC: ",str(auroc_score),"  AP: ",str(ap_score))
  
  auroc_score = roc_auc_score(y_true=truth, y_score=ensemble_posterior_means_var_clipped)  
  ap_score = average_precision_score(y_true=truth, y_score=ensemble_posterior_means_var_clipped)
  print("using ensemble mean of posterior var,  AUROC: ",str(auroc_score),"  AP: ",str(ap_score))
  
  auroc_score = roc_auc_score(y_true=truth, y_score=ensemble_posterior_vars_mean_clipped)  
  ap_score = average_precision_score(y_true=truth, y_score=ensemble_posterior_vars_mean_clipped)
  print("using ensemble var of posterior mean,  AUROC: ",str(auroc_score),"  AP: ",str(ap_score))
  
  auroc_score = roc_auc_score(y_true=truth, y_score=ensemble_posterior_vars_var_clipped)  
  ap_score = average_precision_score(y_true=truth, y_score=ensemble_posterior_vars_var_clipped)
  print("using ensemble var of posterior var,  AUROC: ",str(auroc_score),"  AP: ",str(ap_score))
  
  
  f, axes = plt.subplots(6, 2, figsize=(20, 20))
  
  bins0 = np.linspace(-100.0,150.0,250)
  axes[0,0].hist(ensemble_elbo_mean[:492])
  axes[0,0].set_xlabel("ensemble mean of elbo from valid data")
  axes[0,1].hist(ensemble_elbo_mean[492:])
  axes[0,1].set_xlabel("ensemble mean of elbo from anomlies")
  
  bins1 = np.linspace(0.0,8000,1000)
  axes[1,0].hist(ensemble_elbo_var[:492])
  axes[1,0].set_xlabel("ensemble var of elbo from valid data")
  axes[1,1].hist(ensemble_elbo_var[492:])
  axes[1,1].set_xlabel("ensemble var of elbo from anomlies")
  
  bins2 = np.linspace(-1.0,1.0,100)
  axes[2,0].hist(ensemble_posterior_means_mean[:492])
  axes[2,0].set_xlabel("ensemble mean of  posterior mean from valid data")
  axes[2,1].hist(ensemble_posterior_means_mean[492:])
  axes[2,1].set_xlabel("ensemble mean of  posterior mean from anomalies")
  bins3 = np.linspace(0,5,100)
  axes[3,0].hist(ensemble_posterior_means_var[:492])
  axes[3,0].set_xlabel("ensemble var of  posterior mean from valid data")
  axes[3,1].hist(ensemble_posterior_means_var[492:])
  axes[3,1].set_xlabel("ensemble var of  posterior mean from anomalies")
  
  bins4 = np.linspace(0,10,100)
  axes[4,0].hist(ensemble_posterior_vars_mean[:492])
  axes[4,0].set_xlabel("ensemble mean of  posterior var from valid data")
  axes[4,1].hist(ensemble_posterior_vars_mean[492:])
  axes[4,1].set_xlabel("ensemble mean of  posterior var from anomalies")
  bins5 = np.linspace(0,100,1000)
  axes[5,0].hist(ensemble_posterior_vars_var[:492])
  axes[5,0].set_xlabel("ensemble var of  posterior var from valid data")
  axes[5,1].hist(ensemble_posterior_vars_var[492:])
  axes[5,1].set_xlabel("ensemble var of  posterior var from anomalies")
  
  
  '''
  
  ensemble_posterior_vars_mean_thresholds = np.arange(1, 2.2, 0.02)
  print("using ensemble mean of posterior var")
  print_stats(ensemble_posterior_vars_mean, truth, ensemble_posterior_vars_mean_thresholds, "ensemble mean of post var")
  
  ensemble_posterior_means_var_thresholds = np.arange(0.3, 1.5, 0.1)
  print("using ensemble var of posterior mean")
  print_stats(ensemble_posterior_means_var, truth, ensemble_posterior_means_var_thresholds, "ensemble var of post mean")
  '''
  
  
  '''
  f, axes = plt.subplots(4, 2, figsize=(20, 20))
  bins1 = np.linspace(-2.0,1.0,50)
  axes[0,0].hist(ensemble_posterior_means_mean[:492],bins1)
  axes[0,0].set_xlabel("ensemble mean of  posterior mean from valid data")
  axes[0,1].hist(ensemble_posterior_means_mean[492:],bins1)
  axes[0,1].set_xlabel("ensemble mean of  posterior mean from anomalies")
  bins2 = np.linspace(0,100,100)
  axes[1,0].hist(ensemble_posterior_means_var[:492], bins2)
  axes[1,0].set_xlabel("ensemble var of  posterior mean from valid data")
  axes[1,1].hist(ensemble_posterior_means_var[492:],bins2)
  axes[1,1].set_xlabel("ensemble var of  posterior mean from anomalies")
  
  bins3 = np.linspace(0,100,100)
  axes[2,0].hist(ensemble_posterior_vars_mean[:492],bins3)
  axes[2,0].set_xlabel("ensemble mean of  posterior var from valid data")
  axes[2,1].hist(ensemble_posterior_vars_mean[492:],bins3)
  axes[2,1].set_xlabel("ensemble mean of  posterior var from anomalies")
  bins4 = np.linspace(0,6000,600)
  axes[3,0].hist(ensemble_posterior_vars_var[:492],bins4)
  axes[3,0].set_xlabel("ensemble var of  posterior var from valid data")
  axes[3,1].hist(ensemble_posterior_vars_var[492:],bins4)
  axes[3,1].set_xlabel("ensemble var of  posterior var from anomalies")
  
  '''
    
if __name__ == "__main__":
  tf.app.run()

In [0]:
def _softplus_inverse(x):
  """Helper which computes the function inverse of `tf.nn.softplus`."""
  return tf.log(tf.expm1(x))

def make_encoder(activation, latent_size, base_depth):
  """Create the encoder function.
  Args:
    activation: Activation function to use.
    latent_size: The dimensionality of the encoding.
    base_depth: The lowest depth for a layer.
  Returns:
    encoder: A `callable` mapping a `Tensor` of images to a
      `tf.distributions.Distribution` instance over encodings.
  """
  conv = functools.partial(
      tf.keras.layers.Conv2D, padding="SAME", activation=activation)

  encoder_net = tf.keras.Sequential([
      conv(base_depth, 5, 1),
      conv(base_depth, 5, 2),
      conv(2 * base_depth, 5, 1),
      conv(2 * base_depth, 5, 2),
      conv(4 * latent_size, 7, padding="VALID"),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(2 * latent_size, activation=None),
  ])

  def encoder(images):
    images = 2 * tf.cast(images, dtype=tf.float32) - 1
    net = encoder_net(images)
    return tfd.MultivariateNormalDiag(
        loc=net[..., :latent_size],
        scale_diag=tf.nn.softplus(net[..., latent_size:] +
                                  _softplus_inverse(1.0)),
        name="code")

  return encoder


def make_decoder(activation, latent_size, output_shape, base_depth):
  """Create the decoder function.
  Args:
    activation: Activation function to use.
    latent_size: Dimensionality of the encoding.
    output_shape: The output image shape.
    base_depth: Smallest depth for a layer.
  Returns:
    decoder: A `callable` mapping a `Tensor` of encodings to a
      `tf.distributions.Distribution` instance over images.
  """
  deconv = functools.partial(
      tf.keras.layers.Conv2DTranspose, padding="SAME", activation=activation)
  conv = functools.partial(
      tf.keras.layers.Conv2D, padding="SAME", activation=activation)

  # first filter has size 7 for mnist, 8 for cifar (32x32)
  filter_width = 7 if output_shape[0] == 28 else 8
  
  decoder_net = tf.keras.Sequential([
      deconv(2 * base_depth, filter_width, padding="VALID"),
      deconv(2 * base_depth, 5),
      deconv(2 * base_depth, 5, 2),
      deconv(base_depth, 5),
      deconv(base_depth, 5, 2),
      deconv(base_depth, 5),
      conv(output_shape[-1], 5, activation=None),
  ])

  def decoder(codes):
    original_shape = tf.shape(codes)
    # Collapse the sample and batch dimension and convert to rank-4 tensor for
    # use with a convolutional decoder network.
    codes = tf.reshape(codes, (-1, 1, 1, latent_size))
    logits = decoder_net(codes)
    logits = tf.reshape(
        logits, shape=tf.concat([original_shape[:-1], output_shape], axis=0))
    return tfd.Independent(
        tfd.Bernoulli(logits=logits),
        reinterpreted_batch_ndims=len(output_shape),
        name="image")

  return decoder




def model_fn(features, labels, mode, params, config):
  """Build the model function for use in an estimator.
  Arguments:
    features: The input features for the estimator.
    labels: The labels, unused here.
    mode: Signifies whether it is train or test or predict.
    params: Some hyperparameters as a dictionary.
    config: The RunConfig, unused here.
  Returns:
    EstimatorSpec: A tf.estimator.EstimatorSpec instance.
  """
  del labels, config
  predictions = {}

  if params["analytic_kl"] and params["latent_size"] != 1:
    raise NotImplementedError(
        "Using `analytic_kl` is only supported when `mixture_components = 1` "
        "since there's no closed form otherwise.")
    
  encoder = make_encoder(params["activation"],
                         params["latent_size"],
                         params["base_depth"])
  
  image_shape = features.get_shape().as_list()[1:]
  decoder = make_decoder(params["activation"],
                         params["latent_size"],
                         image_shape,
                         params["base_depth"])
  if params['use_NF']:
    latent_prior = make_NF_prior(params["latent_size"],params["n_flows"])
  else:
    latent_prior = make_mixture_prior(params["latent_size"],
                                      params["mixture_components"])

  image_tile_summary("input", tf.to_float(features), rows=1, cols=16)

  approx_posterior = encoder(features)
  approx_posterior_sample = approx_posterior.sample(params["n_samples"])
  decoder_likelihood = decoder(approx_posterior_sample)
  image_tile_summary(
      "recon/sample",
      tf.to_float(decoder_likelihood.sample()[:3, :16]),
      rows=3,
      cols=16)
  image_tile_summary(
      "recon/mean",
      decoder_likelihood.mean()[:3, :16],
      rows=3,
      cols=16)

  # `distortion` is just the negative log likelihood.
  distortion = -decoder_likelihood.log_prob(features)
  avg_distortion = tf.reduce_mean(distortion)
  tf.summary.scalar("distortion", avg_distortion)
  
  
  if params["analytic_kl"]:
    raise ValueError('Not Completely Implemented!')
    rate = tfd.kl_divergence(approx_posterior, latent_prior)
  else:
    rate = (approx_posterior.log_prob(approx_posterior_sample)
            - latent_prior.log_prob(approx_posterior_sample))
  avg_rate = tf.reduce_mean(rate)
  tf.summary.scalar("rate", avg_rate)

  elbo_local = -(rate + distortion)

  elbo = tf.reduce_mean(elbo_local)
  loss = -elbo
  tf.summary.scalar("elbo", elbo)
  
  # negative log-likelihood of encoded inputs under likelihood model p(x|z)
  # lower is better
  predictions['distortion'] = distortion 
  predictions['rate'] = rate
  predictions['elbo'] = elbo_local

  importance_weighted_elbo = tf.reduce_mean(
      tf.reduce_logsumexp(elbo_local, axis=0) -
      tf.log(tf.to_float(params["n_samples"])))
  tf.summary.scalar("elbo/importance_weighted", importance_weighted_elbo)

  # Decode samples from the prior for visualization.
  random_image = decoder(latent_prior.sample(16))
  image_tile_summary(
      "random/sample", tf.to_float(random_image.sample()), rows=4, cols=4)
  image_tile_summary("random/mean", random_image.mean(), rows=4, cols=4)

  # Perform variational inference by minimizing the -ELBO.
  global_step = tf.train.get_or_create_global_step()
  learning_rate = tf.train.cosine_decay(params["learning_rate"], global_step,
                                        params["max_steps"])
  tf.summary.scalar("learning_rate", learning_rate)
  optimizer = tf.train.AdamOptimizer(learning_rate)
  train_op = optimizer.minimize(loss, global_step=global_step)

  # estimator predictions for inference + visualization
  predictions['approx_posterior_mean'] = approx_posterior.mean()
  predictions['approx_posterior_stddev'] = approx_posterior.scale.diag
  
  # adversarial perturbation
  grad, = tf.gradients(loss, features)
  adversarial_example = features - .1 * tf.sign(grad) # optimize the gibberish to minimize loss.
  predictions['adversarial_example'] = adversarial_example


    
  return tf.estimator.EstimatorSpec(
      mode=mode,
      loss=loss,
      train_op=train_op,
      eval_metric_ops = {
          "elbo": tf.metrics.mean(elbo),
          "elbo/importance_weighted": tf.metrics.mean(importance_weighted_elbo),
          "rate": tf.metrics.mean(avg_rate),
          "distortion": tf.metrics.mean(avg_distortion),},
      predictions=predictions
  )




In [0]:
def main(argv):
  
  M=1 # number of models in ensemble
  for i in range(M):
    params = FLAGS.flag_values_dict()
    params["activation"] = getattr(tf.nn, params["activation"])

    #FLAGS.model_dir = "/usr/local/google/home/ejang/tmp/cifar10/vae%d" % i
    FLAGS.model_dir = "drive/Colab Notebooks/cifar10/vae%d" % i

    if FLAGS.delete_existing and tf.gfile.Exists(FLAGS.model_dir):
      tf.logging.warn("Deleting old log directory at {}".format(FLAGS.model_dir))
      tf.gfile.DeleteRecursively(FLAGS.model_dir)
    tf.gfile.MakeDirs(FLAGS.model_dir)

    train_input_fn, eval_input_fn = get_dataset('cifar10', FLAGS.batch_size)

    estimator = tf.estimator.Estimator(
        model_fn,
        params=params,
        config=tf.estimator.RunConfig(
            model_dir=FLAGS.model_dir,
            save_checkpoints_steps=FLAGS.viz_steps,
        ),
    )

    for _ in range(FLAGS.max_steps // FLAGS.viz_steps):
      estimator.train(train_input_fn, steps=FLAGS.viz_steps)
      eval_results = estimator.evaluate(eval_input_fn)
      print("Evaluation_results:\n\t%s\n" % eval_results)
    

if __name__ == "__main__":
  tf.app.run()
    

In [0]:
params = FLAGS.flag_values_dict()
params["activation"] = getattr(tf.nn, params["activation"])

def eval_ensemble(input_fn):
  model_results = []
  adversarial_examples = []
  for i in range(5):
    #FLAGS.model_dir = "/usr/local/google/home/ejang/tmp/fashion_mnist/vae%d" % i
    #FLAGS.model_dir ="drive/Colab Notebooks/fashion_mnist/vae%d" % i
    FLAGS.model_dir ="drive/Colab Notebooks/creditcard/vae%d" % i
    print('Evaluating eval samples for %s' % FLAGS.model_dir)
    assert tf.train.latest_checkpoint(FLAGS.model_dir) is not None
    
    estimator = tf.estimator.Estimator(
        #model_fn,
        anomaly_model_fn,
        params=params,
        config=tf.estimator.RunConfig(
            model_dir=FLAGS.model_dir,
        )
    )
    batch_results_ = list(estimator.predict(
        input_fn,
        predict_keys=['rate', 'distortion', 'elbo', 'elbo_local_mean'],
        yield_single_examples=False
    ))
    
    elbos = np.concatenate([b['elbo'].T for b in batch_results_], axis=0)
    elbos = np.sum(elbos, axis=1)

    distortions = np.concatenate([b['distortion'].T for b in batch_results_], axis=0)
    distortions = np.sum(distortions, axis=1)

    rates = np.concatenate([b['rate'].T for b in batch_results_], axis=0)
    rates = np.sum(rates, axis=1)
   
    
    model_results.append((rates, distortions, elbos))
    
  return np.stack(model_results)

In [0]:
# Plot Rate-Distortion Curve for true data (good model minimizes them both)

def plot_rd(model_results):
  f, axes = plt.subplots(1, 2, figsize=(10, 5))

  axes[0].scatter(model_results[0, 0, :], model_results[0, 1, :])
  axes[0].set_xlabel('Rate')
  axes[0].set_ylabel('Distortion')
  axes[1].hist(model_results[0, 2, :])
  axes[1].set_xlabel('ELBO')
  
def plot_rd2(model_results):
  f, axes = plt.subplots(1, 2, figsize=(10, 5))

  axes[0].scatter(model_results[0, 0, :], model_results[0, 1, :])
  axes[0].set_xlabel('Rate')
  axes[0].set_ylabel('Distortion')
  axes[0].set_xlim(right=600)
  axes[0].set_ylim(bottom=-5000)
  axes[1].hist(model_results[0, 2, :])
  axes[1].set_xlabel('ELBO')
  

In [0]:

# Ensemble Statistics
def plot_ensemble_stats(model_results):
  f, axes = plt.subplots(1, 3, figsize=(15, 5))

  for i, name in enumerate(['Rate', 'Distortion', 'ELBO']):
    mean = np.mean(model_results[:, i, :], axis=0)
    var = np.var(model_results[:, i, :], axis=0)
    axes[i].scatter(mean, var)
    axes[i].set_xlabel('Ensemble %s mean' % name)
    axes[i].set_ylabel('Ensemble %s variance' % name)
    
def plot_ensemble_stats2(model_results):
  f, axes = plt.subplots(1, 3, figsize=(15, 5))

  mean = np.mean(model_results[:, 0, :], axis=0)
  var = np.var(model_results[:, 0, :], axis=0)
  axes[0].scatter(mean, var)
  axes[0].set_xlabel('Ensemble %s mean' % 'Rate')
  axes[0].set_ylabel('Ensemble %s variance' % 'Rate')
  axes[0].set_xlim(right=600)
  axes[0].set_ylim(top=10000)
  
  mean = np.mean(model_results[:, 1, :], axis=0)
  var = np.var(model_results[:, 1, :], axis=0)
  axes[1].scatter(mean, var)
  axes[1].set_xlabel('Ensemble %s mean' % 'Distortion')
  axes[1].set_ylabel('Ensemble %s variance' % 'Distortion')
  axes[1].set_xlim(left=-4000)
  axes[1].set_ylim(top=200000)
  
  mean = np.mean(model_results[:, 2, :], axis=0)
  var = np.var(model_results[:, 2, :], axis=0)
  axes[2].scatter(mean, var)
  axes[2].set_xlabel('Ensemble %s mean' % 'ELBO')
  axes[2].set_ylabel('Ensemble %s variance' % 'ELBO')
  axes[2].set_xlim(right=0)
  axes[2].set_ylim(bottom=-100000,top=100000)
  
                     

In [0]:
_, eval_input_fn = get_data('fashion_mnist', FLAGS.batch_size)
true_model_results_ = eval_ensemble(eval_input_fn)
plot_rd(true_model_results_)
plot_ensemble_stats(true_model_results_)

In [0]:
# A quick test to make sure that we can do gradient descent computation is done correctly

g = tf.Graph()
with g.as_default():
  x = tf.placeholder(tf.float32, (32, 28, 28, 1))
  params = FLAGS.flag_values_dict()
  params["activation"] = getattr(tf.nn, params["activation"])

  espec = model_fn(x, 0, tf.estimator.ModeKeys.PREDICT, params, None)
  adv_example = espec.predictions['adversarial_example']
  saver = tf.train.Saver()
  adversarial_batches = []
  with tf.train.MonitoredSession() as sess:
    saver.restore(sess, tf.train.latest_checkpoint("/usr/local/google/home/ejang/tmp/fashion_mnist/vae%d" % 0))
    for b in range(10):
      # A batch of adversarial examples
      adv_example_ = np.random.normal(size=(32, 28, 28, 1))
      # for i in range(3):
      e_, adv_example_ = sess.run([espec.predictions['elbo'], adv_example], {x: adv_example_})
      e_, _ = sess.run([espec.predictions['elbo'], adv_example], {x: adv_example_})
      print('%d adversarial ELBO: %f' % (b, np.mean(e_)))
      adversarial_batches.append(adv_example_)
adversarial_inputs_ = np.concatenate(adversarial_batches, axis=0)

In [0]:
# construct an input_fn from the adversarial examples
def build_adversarial_input_fn(adversarial_inputs, batch_size):
  dummy_label = 0
  # just examine first model's adversarial inputs
  dataset = tf.data.Dataset.from_tensor_slices(adversarial_inputs).map(
          lambda feature: (feature, dummy_label)).batch(batch_size)
  eval_input_fn = lambda: dataset.make_one_shot_iterator().get_next()
  return eval_input_fn

adversarial_input_fn = build_adversarial_input_fn(adversarial_inputs_, FLAGS.batch_size)
adv_batch_results_ = eval_ensemble(adversarial_input_fn)
plot_rd(adv_batch_results_)
plot_ensemble_stats(adv_batch_results_)